## Context-free grammars and CKY parsing

## Load grammar, data and pre-processing


### The CNF Grammar file


First we read the grammar file provided to us and check all the details of the grammar that we need in this assignment.

In [8]:
'''
We will use NLTK.CFG library to read the CNF grammar file 
provided to us for this assignment.
'''

import nltk
filepath_grammar = 'atis/atis-grammar-cnf.cfg'
file = open(filepath_grammar, 'r', encoding='utf-8')

In [9]:
'''
We read the grammar file here
'''

grammar = CFG.fromstring(file)
grammar

<Grammar with 20326 productions>

In [10]:
'''
Here we check the starting symbol of the grammar.
We also check the data type of the symbols in the grammar. We will need this in the CYK algorithm later on.
'''

print('Starting symbol:', grammar.start())
print('Data type of the symbol:', type(grammar.start()))

Starting symbol: SIGMA
Data type of the symbol: <class 'nltk.grammar.Nonterminal'>


In [11]:
'''
We read the production rules of the grammar and print some of them to check the rules.
'''

productions = grammar.productions()
for rule in productions[:10]:
    print(rule)

SIGMA -> ADJ_ABL JYL
GCC -> AVP_RB GCD
GCD -> NP_DTS pt_char_per
GCA -> AVP_RB GCB
GCB -> COMPCL_VBZ pt_char_per
GCK -> NP_NNS GCL
GCL -> AJP_JJ pt_char_per
GCI -> NP_DTS GCJ
GCJ -> AJP_JJR pt_char_per
GCG -> NP_DT GCH


### The ATIS test data

Here we get the ATIS test data file

In [12]:
'''
We read the ATIS sentences and parse them into a readable format 
with each sentence with its number of parse trees.
'''

atis_sents = nltk.parse.util.extract_test_sentences(nltk.data.load('grammars/large_grammars/atis_sentences.txt'))

In [73]:
'''
Check some sample sentences in the ATIS data set
'''

atis_sents[15]

(['can',
  'you',
  'tell',
  'me',
  'about',
  'the',
  'flights',
  'from',
  'saint',
  'petersburg',
  'to',
  'toronto',
  'again',
  '.'],
 3)

## Code and Implementation


### Node class (for backpointers and parse trees):
We will be using this Node class to instantiate each node for holding backpointers to the previous node. This will help us in creating the parse trees later on.

In [14]:
'''
Node class definition
'''
class Node:

    '''
    Constructor method which initializes the root,
    left child, right child, terminal and the status
    of the node.
    '''
    def __init__(self, root, left, right, end):
        self._root = root
        self._left = left
        self._right = right
        self._terminal = end
        self._status = True
        if end == None:
            self._status = False

    '''
    Class property root that returns the root of the node
    '''
    @property
    def root(self):
        return self._root

    '''
    Class property left that returns the left child of the node
    '''
    @property
    def left(self):
        return self._left

    '''
    Class property right that returns the right child of the node
    '''
    @property
    def right(self):
        return self._right

    '''
    Class property status that returns the status of the node
    '''
    @property
    def status(self):
        return self._status

    '''
    Class property terminal that returns the terminal of the node
    '''
    @property
    def terminal(self):
        return self._terminal

### Function getTrees(nodes_back):
This function takes the **[0][n] cell of the backpointers** chart (returned from the CKY algorithm) as input. Now, using those backpointers, we backtrack and find the parse trees using the **getAllParseTrees(root, indent)** function recursively.

In [15]:
'''
getTrees(back_nodes) definition:
'''
def getTrees(back_nodes):
    
    INDENT = 3
    '''
    Check boolean variable initially set to False
    '''
    check = False
    
    '''
    Initialize the count of parse trees to 0
    '''
    countOfParseTrees = 0
    
    '''
    For each of the nodes in the backpointer nodes,
    we check if its a root node (SIGMA). If yes, then
    we print it get all the parse trees for that recursively
    using the getAllParseTrees(node) function and set the 
    check varuable to True.
    '''
    for node in back_nodes:
        '''
        Check if the node is root node (SIGMA)
        '''
        if node.root == grammar.start():
            print(node.root)
            '''
            This will be a recursive call inside the function
            '''
            print(getAllParseTrees(node, INDENT))
            print()
            countOfParseTrees += 1
            check = True

    '''
    If the check variable is still False, then the given
    sentence is not in the given grammar.
    '''
    if not check:
        print('The sentence is not in the grammar!')
    return countOfParseTrees

### Function getAllParseTrees(root):
This is the function which actually is called recursively inside its body. This takes root of the tree as input. We call this function **recursively** to print the parse trees for the given sentence according to the given grammar.

In [16]:
'''
getAllParseTrees(root) definition:
'''
def getAllParseTrees(root, indent):
    
    '''
    Printing the tree at a specific level where there are 2 nodes
    '''
    if root.status:
        return '(' + str(root.root) + ' ' + root.terminal + ')'
    
    '''
    Left subtree of the root input node
    '''
    new1 = indent + 2 + len(str(root.left.root))
    
    '''
    Right subtree of the root input node
    '''
    new2 = indent + 2 + len(str(root.right.root))
    
    '''
    This prints the left subtree of the root input node
    '''
    left = getAllParseTrees(root.left, new1)
    
    '''
    This prints the right subtree of the root input node
    '''
    right = getAllParseTrees(root.right, new2)
    
    '''
    Printing the tree at a specific level where there are 3 nodes
    '''
    return '(' + str(root.root) + ' ' + left + '\n' + ' '*indent + right + ')'

### CKY algorithm / CYK algorithm
This is the implementation of the CKY algorithm (sometimes also known as the CYK algorithm). All the details of the code have been specifically mentioned in the comments below.

In [33]:
'''
Function cky(grammar, sentence) definition
- takes the CNF-form grammar and a sentence as inputs
'''
def cky(grammar, sentence):
    
    n = len(sentence)
    
    '''
    Here we create the chart of POS tags and the chart of the backpointers
    with n + 1 rows and columns each. Each of the cell is a set().
    '''
    chart = [[set() for i in range(n + 1)] for i in range(n + 1)]
    nodes_back = [[set() for i in range(n + 1)] for j in range(n + 1)]
    
    '''
    Here we initialize the diagonal elements of the chart with the single terminal 
    rules of the words in the sentence. Also, we initialize the diagonal backpointers
    with the Non-Terminal of the rule.
    '''
    for i in range(n):
        for production in grammar.productions(rhs=sentence[i]):
                chart[i][i + 1].add(production.lhs())
                nodes_back[i][i + 1].add(Node(production.lhs(), None, None, sentence[i]))

    '''
    This is the main implementation of the CKY/CYK algorithm.
    
    COPYRIGHT DISCLAIMER:
    ---------------------
    This implementation is inspired by Professor Koller's sample implementation 
    of the CKY algorithm from the lecture slides of University of Potsdam 
    which can be found here: http://www.ling.uni-potsdam.de/tcl/clt14/vorlesungen/05%20CKY-Parsing.pdf
    
    However, I have just taken inspiration of the code and not copied it. I have implemented the code myself.
    I have also implemented the backpointers logic by myself completely.
    '''
    for width in range(2, n + 1):
        for i in range(0, n - width + 1):
            for j in range(1, width):
                nts1, nts2 = chart[i][i + j], chart[i + j][i + width]
                for nt1 in nts1:
                    productions = grammar.productions(rhs=nt1)
                    for production in productions:
                        if production.rhs()[1] in nts2:
                            chart[i][i + width].add(production.lhs())
                            
                            '''
                            Here we update the corresponding backpointer chart for the corresponding 
                            update in the POS tag chart.
                            '''
                            for b in nodes_back[i][i + j]:
                                for c in nodes_back[i + j][i + width]:
                                    if b.root == production.rhs()[0] and c.root == production.rhs()[1]:
                                        nodes_back[i][i + width].add(Node(production.lhs(), b, c, None))

    print()
    
    '''
    Here, the CKY algorithm just tells whether an input sentence
    is in the given input grammar or not.
    '''
    START_SYMBOL = nltk.grammar.Nonterminal('SIGMA')
    if START_SYMBOL in chart[0][n]:
        print('The sentence is in the grammar!')
    else:
        print('The sentence is not in the grammar!')
    print()
    
    '''
    We return the chart as well as the node [0][n] of the backpointers chart which
    we will use later on to print the parse trees.
    '''
    return chart, nodes_back[0][n]

### Testing the CKY algorithm
We here test our implementation of the CKY algorithm with some sample sentences. Here are the sample sentences:


1. **prices .**  - (proper sentence in the grammar)
2. **the school is closed .** - (proper sentence but not in the grammar)
3. **flights please .** - (proper sentence in the grammar)
4. **closed flight is .** - (Ungrammatical sentence)

In [18]:
'''
All the sample sentences
'''
sentences = [
    ['prices', '.'],
    ['the', 'school', 'is', 'closed', '.'],
    ['flights', 'please', '.'],
    ['closed', 'flight', 'is', '.']
           ]
'''
Run the CKY algorithm for each of the sentence
'''
for sentence in sentences:
    '''
    Print the chart to check the output.
    '''
    print('SENTENCE:', ' '.join(sentence))
    chart, backpointers = cky(grammar, sentence)
    print('------------------------------------------------------------------------------------------------')
    for i in range(len(chart)):
        for j in range(len(chart[0])):
            print('|', chart[i][j], end='')
        print()
        print('------------------------------------------------------------------------------------------------')
    print()
    print()

SENTENCE: prices .

The sentence is in the grammar!

------------------------------------------------------------------------------------------------
| set()| {VP_VBZ, pt207, AVPNP_NNS, SIGMA, VERB_VBZ, NOUN_NNS, NP_NNS}| {GNN, BID, KBW, BJJ, KAN, JIB, DPR, GRP, AQP, GLH, AJV, GIT, JIP, BHB, DAF, GUJ, DZA, DBA, GIJ, GBN, JJL, DOE, DON, GJF, GDR, DLW, BFZ, HYH, GOB, CZT, JJD, AYD, GLZ, GSH, HFD, KCD, GAF, CSP, JZK, HVH, EZX, AXJ, KAT, CTK, KCJ, GFD, KDT, CWW, JSB, UQ, GSN, BRR, ES, EEI, CXI, GXB, GYR, JIR, BOT, EEU, JZY, SI, KDH, HYJ, CXR, GHT, RY, JJP, BQH, GIN, IQ, GWH, GAP, DAL, GRB, CRF, KAL, GCR, GBB, DECL_VBZ, JKB, DZJ, DPF, RE, SIGMA, DRB, BQL, CZZ, NP_NNS, GCF, GFN, DXT, GBV, GXX, DDI, HFB, GJZ}
------------------------------------------------------------------------------------------------
| set()| set()| {pt_char_per}
------------------------------------------------------------------------------------------------
| set()| set()| set()
------------------------------------------

### Ungrammatical sentences

Here are some ungrammatical sentences that we tried to feed to the CKY algorithm:


1. **. prices** --> Result: **NO**
2. **flights . please** --> Result: **YES** *(Surprising!)*
3. **closed flight is .** --> Result: **NO**
4. **are cancel flight .** --> Result: **NO**
5. **cancel is flight** --> Result: **NO**

In [19]:
'''
All the sample sentences
'''
sentences = [
    ['.', 'prices'],
    ['flights', '.', 'please'],
    ['closed', 'flight', 'is', '.'],
    ['are', 'cancel', 'flight', '.'],
    ['cancel', 'is', 'flight']
           ]
'''
Run the CKY algorithm for each of the sentence
'''
for sentence in sentences:
    '''
    Print the chart to check the output.
    '''
    print('SENTENCE:', ' '.join(sentence))
    chart, backpointers = cky(grammar, sentence)
    print('------------------------------------------------------------------------------------------------')
    for i in range(len(chart)):
        for j in range(len(chart[0])):
            print('|', chart[i][j], end='')
        print()
        print('------------------------------------------------------------------------------------------------')
    print('\n\n================================================================================================\n\n')

SENTENCE: . prices

The sentence is not in the grammar!

------------------------------------------------------------------------------------------------
| set()| {pt_char_per}| set()
------------------------------------------------------------------------------------------------
| set()| set()| {VP_VBZ, pt207, AVPNP_NNS, SIGMA, VERB_VBZ, NOUN_NNS, NP_NNS}
------------------------------------------------------------------------------------------------
| set()| set()| set()
------------------------------------------------------------------------------------------------




SENTENCE: flights . please

The sentence is in the grammar!

------------------------------------------------------------------------------------------------
| set()| {VP_VBZ, pt207, AVPNP_NNS, SIGMA, VERB_VBZ, NOUN_NNS, NP_NNS}| {GNN, BID, KBW, BJJ, KAN, JIB, DPR, GRP, AQP, GLH, AJV, GIT, JIP, BHB, DAF, GUJ, DZA, DBA, GIJ, GBN, JJL, DOE, DON, GJF, GDR, DLW, BFZ, HYH, GOB, CZT, JJD, AYD, GLZ, GSH, HFD, KCD, GAF, CSP, 

### Test the parse tree printing algorithm


Sample sentence: **prices .**


Number of total parse trees: 2

In [20]:
sample_sentence = ['prices', '.']
chart, backpointers = cky(grammar, sample_sentence)
countOfParseTrees = getTrees(backpointers)

print('Number of total parse trees: ', countOfParseTrees)


The sentence is in the grammar!

SIGMA
(SIGMA (NOUN_NNS prices)
   (pt_char_per .))

SIGMA
(SIGMA (VERB_VBZ prices)
   (pt_char_per .))

Number of total parse trees:  2


### Some sentences with less than 5 parse trees:

We are going to use 5 sentences which have less than 5 total parse trees according to the ATIS CNF Grammar. The sentences are as follows:


1. **what is the fare .** --> Number of parse trees: 2
2. **can you tell me about the flights from saint petersberg to toronto again .** --> Number of parse trees: 3
3. **show availability .** --> Number of parse trees: 3
4. **show the flights .** --> Number of parse trees: 2
5. **what is the cheapest ticket from memphis to miami .** --> Number of parse trees: 5

In [21]:
'''
All the sample sentences
'''
sentences = [
    ['what', 'is', 'the', 'fare', '.'],
    ['can', 'you', 'tell', 'me', 'about', 'the', 'flights', 'from', 'saint', 'petersburg', 'to', 'toronto', 'again', '.'],
    ['show', 'availability', '.'],
    ['show', 'the', 'flights', '.'],
    ['what', 'is', 'the', 'cheapest', 'ticket', 'from', 'memphis', 'to', 'miami', '.']
           ]
'''
Run the CKY algorithm for each of the sentence
'''
for sentence in sentences:
    print('SENTENCE: ', ' '.join(sentence))
    chart, backpointers = cky(grammar, sentence)
    countOfParseTrees = getTrees(backpointers)
    print('Number of total parse trees: ', countOfParseTrees)
    print('\n\n=======================================================\n\n')
    

SENTENCE:  what is the fare .

The sentence is in the grammar!

SIGMA
(SIGMA (NP_DT what)
   (GDO (VERB_BEZ is)
        (GDP (NP_NN (ADJ_AT the)
                    (NOUN_NN fare))
             (pt_char_per .))))

SIGMA
(SIGMA (NP_DT what)
   (KGC (VERB_BEZ is)
        (NP_NN (ADJ_AT the)
               (JUO (NOUN_NN fare)
                    (pt_char_per .)))))

Number of total parse trees:  2




SENTENCE:  can you tell me about the flights from saint petersburg to toronto again .

The sentence is in the grammar!

SIGMA
(SIGMA (VERB_MD can)
   (BJG (NP_PPSS you)
        (BJH (VERB_VB tell)
             (BJI (NP_PPO me)
                  (BJJ (NP_NNS (AVP_RB (AVP_RB about)
                                       (ADV_RB the))
                               (KAS (NOUN_NNS flights)
                                    (PP_NP (PREP_IN from)
                                           (ITY (NP_NP saint)
                                                (ITZ (NOUN_NP petersburg)
               

### Running the code on all the 98 ATIS test sentences:

In [22]:
import pandas as pd
sentences, countOfTrees = [], []

In [74]:
for sentence in atis_sents[15:16]:      
    
    complete_sentence = ' '.join(sentence[0])
    sentences.append(complete_sentence)
    
    chart, backpointers = cky(grammar, sentence[0])
    countOfParseTrees = getTrees(backpointers)
    countOfTrees.append(countOfParseTrees)


The sentence is in the grammar!

SIGMA
(SIGMA (VERB_MD can)
   (BJG (NP_PPSS you)
        (BJH (VERB_VB tell)
             (BJI (NP_PPO me)
                  (BJJ (NP_NNS (AVP_RB (AVP_RB about)
                                       (ADV_RB the))
                               (KAS (NOUN_NNS flights)
                                    (PP_NP (PREP_IN from)
                                           (KLJ (NOUN_NP (saint saint)
                                                         (petersburg petersburg))
                                                (PP_NP (PREP_IN to)
                                                       (KKY (NOUN_NP toronto)
                                                            (AVP_RB again)))))))
                       (pt_char_per .))))))

SIGMA
(SIGMA (VERB_MD can)
   (BJG (NP_PPSS you)
        (BJH (VERB_VB tell)
             (BJI (NP_PPO me)
                  (BJJ (NP_NNS (AVP_RB (AVP_RB about)
                                       (ADV_RB the))


In [57]:
data = {'Sentence': sentences, '# Parse Trees': countOfTrees}
final_table = pd.DataFrame(data=data)
final_table

,Sentence,# Parse Trees
0,does united flight four seven four slash fourt...,0
1,what kind of aircraft is american 's flight fi...,0
2,show me flights from detroit to san diego on t...,0
3,i 'd like to fly from indianapolis to houston ...,0
4,how much does first class on that flight cost ...,54
5,can you tell me about the flights from saint p...,3
6,i would like to find a flight from charlotte t...,55
7,what if i wanted to leave on may fifth .,0
8,i would like to leave on thursday morning may ...,0
9,how far is it from the airport to the city .,1


In [64]:
writer = pd.ExcelWriter('output.xlsx')
final_table.to_excel(writer,'Sheet1')
writer.save()